In [75]:
import urllib.request

url = "https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py"
urllib.request.urlretrieve(url, "minsearch.py")
print("download successfull")

download successfull


In [76]:
from dotenv import load_dotenv
import os

In [77]:
from google import genai

In [78]:
import minsearch

In [79]:
import json

In [80]:
with open('documents.json','rt') as f_in:
    docs_raw = json.load(f_in)

In [1]:
#docs_raw

In [82]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [83]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [84]:
index = minsearch.Index(
    text_fields = ["question","text","section"],
    keyword_fields = ["course"]
)

In [85]:
q = 'the course is already started can I join now?'

In [86]:
index.fit(documents)

In [87]:
#results

In [88]:
load_dotenv()  # Loads variables from .env file
API_KEY = os.getenv("GEMINI_API_KEY")
#print(API_KEY)


In [89]:
client = genai.Client(api_key=API_KEY)

responses = client.models.generate_content(
    model="gemini-2.0-flash", contents=q
)

In [90]:
responses.text

"Whether you can join a course that has already started depends on a few factors:\n\n*   **The course provider's policy:** Some courses have strict deadlines for enrollment, while others are more flexible. Check the course website or contact the instructor/administrator to find out their specific policy.\n*   **The nature of the course:** Some courses are structured in a way that makes it difficult to catch up if you start late. Others are more self-paced and allow you to jump in at any time.\n*   **How far along the course is:** If the course has just started, it may be easier to catch up than if it's already halfway through.\n*   **Your ability to catch up:** Consider how much time and effort you're willing to put in to catch up on any missed material.\n\nHere's what I recommend you do:\n\n1.  **Find the course website or contact information:** Look for the official website of the course or the contact details of the instructor or administrator.\n2.  **Contact the instructor/administ

In [ ]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [92]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [93]:
def llm(prompt):
    client = genai.Client(api_key=API_KEY)
    responses = client.models.generate_content(
        model="gemini-2.0-flash", contents=prompt)
    return responses

In [94]:
query = "How do I run Kafka?"

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer.text

In [95]:
result = rag(query)
print(result)

There are a few ways to run Kafka, depending on the language you're using:

*   **Java:** In the project directory, run: `java -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java`
*   **Python:**
    1.  Create a virtual environment and run `requirements.txt` in that environment.
    2.  To create a virtual env and install packages (run only once):
        ```
        python -m venv env
        source env/bin/activate
        pip install -r ../requirements.txt
        ```
    3.  To activate it (you'll need to run it every time you need the virtual env):
        ```
        source env/bin/activate
        ```
    4.  To deactivate it:
        ```
        deactivate
        ```

    *   The path for activating the virtual environment on Windows is `env/Scripts/activate`.
    *   The virtual environment should be created only to run the python file and the Docker images should be running first.
*   If you encounter a "Permission denied" error when r

In [96]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [97]:
from elasticsearch import Elasticsearch

In [98]:
es_client = Elasticsearch('http://localhost:9200')

In [99]:
es_client.info()

ObjectApiResponse({'name': 'fd1b4ee9e246', 'cluster_name': 'docker-cluster', 'cluster_uuid': '0vxnMo5AQpiGSA8DrsNJ_Q', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [ ]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

In [101]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [102]:
from tqdm.auto import tqdm

In [103]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [108]:
query = 'I just discovered the course. Can I still join it?'

In [109]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }

    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = [] 
    
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs


In [110]:
def rag(query):
    search_results = elastic_search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer.text

In [111]:
rag(query)

"Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects, so don't leave everything for the last minute.\n"